In [1]:
#import common librarie
from zipfile import ZipFile 
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
from itertools import groupby
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns



In [90]:
file_name = "poems_with_tags.zip"
poem_df = pd.read_csv('poems_with_tags.zip', compression='zip', header=0, quotechar='"')
poem_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']"
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']"
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']"
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']"
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']"


## Aggregate Tags Type

too many tags cause the prediction too wide to predict the type of the poem 

In [10]:
import synonyms

smart_open library not found; falling back to local-filesystem-only
[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/home/lindayang16/anaconda3/lib/python3.8/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/home/lindayang16/anaconda3/lib/python3.8/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/home/lindayang16/anaconda3/lib/python3.8/site-packages/synonyms/data/words.vector.gz] ...


In [11]:
tags_list = poem_df['tags'].tolist()

In [81]:
len(tags_list)

6119

In [12]:
tags_set = set()
for tags in tags_list:
    tag_split_list = tags.strip(';').split(';')
    for tag in tag_split_list:
        if len(tag) > 1:
            tags_set.add(tag)

In [22]:
smaller_tags_set = set()
smaller_tags_set.add('伤情')
error_tags = set()
tags_map = {}
for tag in tags_set:
    if tag not in smaller_tags_set:
        found_similar_tag = False
        for unique_tag in smaller_tags_set:
            if len(synonyms.nearby(tag)[0]) > 0:
                score = synonyms.compare(unique_tag, tag) 
                if score > 0.6:
                    tags_map[tag] = unique_tag
                    found_similar_tag = True
                    break
            else:
                error_tags.add(tag)
                tags_map[tag] = None
                found_similar_tag = True
                print("error added {}".format(tag))
                break
        if not found_similar_tag:
            tags_map[tag] = tag
            smaller_tags_set.add(tag)

error added 感昔伤今
error added 壮志难酬
error added 应制诗
error added 写鬼
error added 高中古诗
error added 忆往
error added 写剑
error added 写花
error added 闺妇
error added 和诗
error added 抒请
error added 赠序
error added 别愁
error added 伤今追昔
error added 迁谪
error added 讽谕
error added 思妇
error added 集句
error added 辞赋精选
error added 酬和
error added 写灯
error added 折柳
error added 吊古
error added 借古寄怀
error added 寻春
error added 叙志
error added 写月
error added 写草
error added 春残
error added 愁怨
error added 乡思
error added 怨刺
error added 离思
error added 愁闷
error added 抚今忆昔
error added 江峡
error added 愁病
error added 骚体赋
error added 写山
error added 赠诗
error added 写景怀古
error added 伤春
error added 闺思
error added 赠友
error added 荐书
error added 写景抒情
error added 寓理
error added 暮归
error added 高中文言文
error added 宋词三百首
error added 托古讽今
error added 写雪
error added 思国
error added 古诗里的十二个月
error added 写物
error added 惜花
error added 咏柳
error added 借古论今
error added 即事
error added 愁恨
error added 寓情于景
error added 伤今感昔
error added 宋词精选
error added 春

/home/lindayang16/anaconda3/lib/python3.8/site-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))


error added 借古抒怀
error added 寓情
error added 惜时
error added 怀今
error added 遣怀
error added 以景结情
error added 边将
error added 忆人
error added 托物言志
error added 写云
error added 思夫
error added 触景感怀
error added 写狗古诗18首
error added 抒志
error added 宫怨
error added 即景抒情
error added 理理想
error added 舟中
error added 咏竹
error added 叙梦
error added 咏剑
error added 抒愤
error added 写茶
error added 写柳
error added 送人
error added 赋别
error added 春愁
error added 讽今
error added 感物兴怀
error added 早教古诗100首
error added 写树


/home/lindayang16/anaconda3/lib/python3.8/site-packages/synonyms/utils.py:246: RuntimeWarning: invalid value encountered in double_scalars
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))


error added 初中古诗
error added 感时
error added 叹恨
error added 感时伤事
error added 写虎
error added 送春
error added 离别｜抒情｜伤感｜怀人
error added 抒情思乡
error added 托志
error added 伤老
error added 怨别
error added 唐诗三百首
error added 追昔
error added 怀友
error added 借凭吊屈
error added 抚事
error added 写桥
error added 写风
error added 写梅
error added 写水
error added 咏古
error added 写马
error added 怨情
error added 写人
error added 托物寄情
error added 怀古伤今
error added 咏志
error added 记梦
error added 写塔
error added 酬赠
error added 咏史怀古
error added 托物抒怀
error added 答别
error added 春天｜写人
error added 邀客
error added 写船
error added 思怀
error added 小学文言文
error added 残春
error added 舒怀
error added 写琴
error added 写竹
error added 初中文言文
error added 寿词
error added 古诗三百首
error added 恨别
error added 以古讽今
error added 思友
error added 对月
error added 以景衬情
error added 惜人
error added 写莲
error added 抚今思昔
error added 看花
error added 怀古伤心


In [14]:
f = open("smaller_tags_set.txt", "w")
for tag in smaller_tags_set:
    f.write(tag+"\n")
f.close()

In [15]:
f = open("error_tags.txt", "w")
for tag in error_tags:
    f.write(tag+"\n")
f.close()

Pre-processed tags name locally into 9 category, then load it back

In [30]:
tags_summary = {}

In [31]:
# Using readlines()
file1 = open('tags.csv', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    split_line = line.split(',')[:2]
    tags_summary[split_line[1]] = split_line[0]
    

file1 = open('error_tags_processed.csv', 'r')
Lines = file1.readlines()
 
count = 0
# Strips the newline character
for line in Lines:
    count += 1
    split_line = line.split(',')[:2]
    print(split_line)
    tags_summary[split_line[1]] = split_line[0]

['政治', '感昔伤今']
['朋友', '怀友']
['政治', '借凭吊屈']
['政治', '抚事']
['政治', '壮志难酬']
['政治', '边将']
['写物', '写桥']
['写物', '写雪']
['写物', '写鬼']
['写物', '写风']
['政治', '思国']
['朋友', '忆人']
['写物', '写梅']
['写物', '写水']
['政治', '托物言志']
['写物', '古诗里的十二个月']
['政治', '忆往']
['写物', '写物']
['写物', '写剑']
['写物', '写云']
['写物', '惜花']
['写物', '写花']
['写物', '咏柳']
['政治', '咏古']
['写物', '写马']
['家庭', '怨情']
['家庭', '闺妇']
['家庭', '思夫']
['写物', '写人']
['政治', '借古论今']
['政治', '即事']
['政治', '触景感怀']
['写物', '写狗古诗18首']
['悲苦', '愁恨']
['政治', '寓情于景']
['政治', '抒志']
['政治', '托物寄情']
['政治', '伤今感昔']
['政治', '托古讽今']
['政治', '怀古伤今']
['朋友', '赠序']
['家庭', '宫怨']
['政治', '春闺']
['朋友', '别愁']
['政治', '伤今追昔']
['政治', '咏志']
['写物', '记梦']
['政治', '抚昔伤今']
['政治', '即景抒情']
['游玩', '迁谪']
['劝勉', '寓人']
['政治', '理理想']
['政治', '讽谕']
['写物', '舟中']
['写物', '写塔']
['朋友', '酬赠']
['写物', '咏竹']
['写物', '月下']
['家庭', '思妇']
['政治', '咏史怀古']
['政治', '托物抒怀']
['政治', '凭吊古迹']
['朋友', '答别']
['写物', '集句']
['写物', '春天｜写人']
['写物', '叙梦']
['写物', '咏剑']
['写物', '写雨']
['政治', '述志']
['政治', '吊古伤今']
['政治', '酬和']
['政治', '抒愤']
['写物', '写茶']


Aggregate all the tags into these 9 labels

In [59]:
processed_tags_summary = {}
for key, synonym in tags_map.items():
    key = key
    if synonym != None:
        synonym = synonym
        if synonym in tags_summary:
            processed_tags_summary[key] = tags_summary[synonym]
        else:
            processed_tags_summary[key] = synonym
    elif key in error_tags:
        if key in tags_summary:
            processed_tags_summary[key] = tags_summary[key]
        else:
            processed_tags_summary[key] = synonym

    

In [111]:
processed_tags_summary['怀人']

'政'

In [114]:
new_tags_list = []
new_first_tag = []
new_columns = {'写物':[], '劝勉':[], '家庭':[], '快乐':[], '悲苦':[], '政治':[], '无':[], '朋友':[], '游玩':[]}
def update_column_value(index, column_names):
    for key, value in new_columns.items():
        value.append(0)
    for column in column_names:
        if column in new_columns.keys():
            new_columns[column][index] = 1
        
for i, tags in enumerate(tags_list):
    tag_split_list = tags.strip(';').split(';')
    new_tags = []
    for tag in tag_split_list:
        if len(tag) > 1:
            if tag in processed_tags_summary:
                new_tag =  processed_tags_summary[tag]
                if new_tag not in new_tags and new_tag != None:
                    new_tags.append(new_tag)
    if new_tags != None: 
        new_tags.sort()
        new_tags_list.append(';'.join(new_tags))
        if len(new_tags) == 0:
            new_first_tag.append('无')    
            update_column_value(i, ['无'])
        else:
            new_first_tag.append(new_tags[0])
            update_column_value(i, new_tags)
            

        

In [118]:
len(new_columns['无'])

6119

In [119]:
poem_df['new_tags'] = new_tags_list
poem_df['new_first_tag'] = new_first_tag
for key, values in new_columns.items():
    poem_df[key] = values

In [127]:
poem_df = poem_df[poem_df['无'] == 0]


In [132]:
poem_df = poem_df.drop(['无'], axis=1)

In [133]:
poem_df.describe()

,star,author_stars,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
count,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000,6107.000000
mean,333.962338,1416.778942,0.421320,0.223023,0.097593,0.135418,0.454560,0.547732,0.211561,0.194858
std,1453.155813,1795.112343,0.493811,0.416308,0.296788,0.342198,0.497972,0.497757,0.408448,0.396124
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.000000,146.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30.000000,609.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,99.000000,1951.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,34273.000000,6155.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [134]:
poem_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,new_tags,new_first_tag,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",政治;游玩,政治,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",写物;劝勉;悲苦;政治,写物,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",写物;政治;游玩,写物,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",劝勉,劝勉,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",写物;快乐;悲苦,写物,1,0,0,1,1,0,0,0


In [135]:
compression_opts = dict(method='zip', archive_name='poems_with_new_tags.csv') 

poem_df.to_csv('poems_with_new_tags.zip', index=False,compression=compression_opts)  